In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split#將資料分成訓練組及測試組
import os
from keras.models import Sequential, load_model

dataframe = pd.read_csv('df_USE_lstm_train.csv',
                         encoding = "Big5",na_filter=True).dropna()

In [ ]:
dataframe

In [ ]:
#########################驗證集

In [ ]:
dataframe_va = pd.read_csv('df_USE_lstm_va.csv',
                         encoding = "Big5",na_filter=True).dropna()

In [ ]:
print(dataframe_va.shape)

dataframe_va=dataframe_va.drop(['TSE 產業別','Lstm','Datamining'], axis=1)#刪除非必要欄位

total_va = dataframe_va.shape  #資料總變數與總資料筆數
total_class_va = dataframe_va.Y_繼續經營疑慮.value_counts()
print(total_va)
print(total_class_va)

In [ ]:
X_va = []
Y_va = []
j_va = 0
for j in range(len(dataframe_va )-5):
    if dataframe_va .iloc[j,0] == dataframe_va .iloc[j+5,0]:  
        X_va.append(dataframe_va.iloc[j:j+4,2:22].values)  #scaled_dataset_
        Y_va.append(dataframe_va.iloc[j+5,22])
    j_va += 1

In [ ]:
Y_va

In [ ]:
Y_va_c =pd.DataFrame(Y_va)
total_class_va_c = Y_va_c.value_counts()
print(total_class_va_c)

In [ ]:
X_va =np.array(X_va)
Y_va =np.array(Y_va)
print(X_va.shape)
print(Y_va.shape)

In [ ]:
##################################END 驗證集

In [ ]:
print(dataframe.shape)

dataframe=dataframe.drop(['TSE 產業別','Lstm','Datamining'], axis=1)#刪除非必要欄位

total = dataframe.shape  #資料總變數與總資料筆數
total_class = dataframe.Y_繼續經營疑慮.value_counts()
print(total)
print(total_class)

In [ ]:
pd.set_option('display.max_columns', None)
dataframe.describe()

In [ ]:
dataframe

In [ ]:
#建立公司名單集合
Name = []
i = 0
for i in range(len(dataframe)):
    Name.append(dataframe.iloc[i,0])
    i += 1
print(len(Name))
print(type(Name))
Nameset = set(Name)
print(len(Nameset))

In [35]:
# window 建立四期觀測資料集
X = []
Y = []
j = 0
for j in range(len(dataframe)-5):
    if dataframe.iloc[j,0] == dataframe.iloc[j+5,0]:  
        X.append(dataframe.iloc[j:j+4,2:22].values)  #scaled_dataset_
        Y.append(dataframe.iloc[j+5,22])
    j += 1

In [36]:
Y_c =pd.DataFrame(Y)
total_class_c = Y_c.value_counts()
print(total_class_c)

1    33661
0     4463
dtype: int64


In [37]:
print(j, len(X), len(Y))
print(type(X))

46704 38124 38124
<class 'list'>


In [38]:
X =np.array(X)
Y =np.array(Y)
print(X.shape)
print(Y.shape)

(38124, 4, 20)
(38124,)


In [39]:
np.set_printoptions(threshold=np.inf)
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [40]:
# #將資料打散，而非照日期排序

def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [41]:
# #將Training Data取一部份當作Test Data
def splitData(X,Y,rate):
  x_train = X[int(X.shape[0]*rate):]
  y_train = Y[int(Y.shape[0]*rate):]
  x_test = X[:int(X.shape[0]*rate)]
  y_test = Y[:int(Y.shape[0]*rate)]
  return x_train, y_train, x_test, y_test

In [42]:

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X, Y)

# split training data and validation data
x_train, y_train, x_test, y_test = splitData(X_train, Y_train, 0.2)

print("資料型別",type(x_train))      #列印陣列資料型別
print("陣列元素資料型別：",x_train.dtype) #列印陣列元素資料型別
print("陣列元素總數：",x_train.size)   #列印陣列尺寸，即陣列元素總數
print("陣列形狀：",x_train.shape)     #列印陣列形狀
print("陣列的維度數目",x_train.ndim)   #列印陣列的維度數目
print("資料型別",type(y_train))      #列印陣列資料型別
print("陣列元素資料型別：",y_train.dtype) #列印陣列元素資料型別
print("陣列元素總數：",y_train.size)   #列印陣列尺寸，即陣列元素總數
print("陣列形狀：",y_train.shape)     #列印陣列形狀
print("陣列的維度數目",y_train.ndim)   #列印陣列的維度數目
# print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

資料型別 <class 'numpy.ndarray'>
陣列元素資料型別： float64
陣列元素總數： 2440000
陣列形狀： (30500, 4, 20)
陣列的維度數目 3
資料型別 <class 'numpy.ndarray'>
陣列元素資料型別： int64
陣列元素總數： 30500
陣列形狀： (30500,)
陣列的維度數目 1


In [43]:
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.constraints import MaxNorm as maxnorm
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import TimeDistributed
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(4, 20), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(32, activation='sigmoid'))
# model.add(Dropout(0.5))
model.add(Dense(16, activation='sigmoid'))
# model.add(Dropout(0.5))
model.add(Dense(8, activation='sigmoid'))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error'
              ,optimizer='adam'
              )
# ,metrics=['accuracy']
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                21760     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 24,513
Trainable params: 24,513
Non-traina

In [44]:

batch_size = 2000
epochs = 2000

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'LSTM_trained_model.h5'

# Use ModelCheckpoint to save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

# earlystop
earlystop = EarlyStopping(monitor='val_loss', patience=300, verbose=1)


cw = {0:0.01 , 1:1}
history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data= (x_test, y_test),
                    class_weight=cw,callbacks=[earlystop, checkpoint])


# loading our save model

print("Loading trained model")
model = load_model(model_path)

# score, acc = model.evaluate(x_test, y_test)

Epoch 1/2000
14/16 [=========================>....] - ETA: 0s - loss: 0.1622
Epoch 1: val_loss improved from inf to 0.17175, saving model to C:\Users\rTie\Desktop\論文\saved_models\LSTM_trained_model.h5
16/16 [==============================] - 3s 54ms/step - loss: 0.1596 - val_loss: 0.1718
Epoch 2/2000
14/16 [=========================>....] - ETA: 0s - loss: 0.1070
Epoch 2: val_loss improved from 0.17175 to 0.14056, saving model to C:\Users\rTie\Desktop\論文\saved_models\LSTM_trained_model.h5
16/16 [==============================] - 0s 29ms/step - loss: 0.1053 - val_loss: 0.1406
Epoch 3/2000
13/16 [=======================>......] - ETA: 0s - loss: 0.0723
Epoch 3: val_loss improved from 0.14056 to 0.12397, saving model to C:\Users\rTie\Desktop\論文\saved_models\LSTM_trained_model.h5
16/16 [==============================] - 0s 25ms/step - loss: 0.0705 - val_loss: 0.1240
Epoch 4/2000
13/16 [=======================>......] - ETA: 0s - loss: 0.0507
Epoch 4: val_loss improved from 0.12397 to 0.115

In [45]:
#在訓練集訓練結果
import math
y_train_predict_0=model.predict(x_train)
y_train_predict_1=y_train_predict_0[:,0]
y_train_predict_2=[np.round(i) for i in y_train_predict_1] 
y_train_predict_3=np.array(y_train_predict_2)
from sklearn import metrics
print("精準度指標：")
print(metrics.classification_report(y_train,y_train_predict_3))
print("混淆矩陣：")
print(metrics.confusion_matrix(y_train,y_train_predict_3))

954/954 [==============================] - 2s 1ms/step
精準度指標：
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3536
           1       0.88      1.00      0.94     26964

    accuracy                           0.88     30500
   macro avg       0.44      0.50      0.47     30500
weighted avg       0.78      0.88      0.83     30500

混淆矩陣：
[[    0  3536]
 [    0 26964]]


C:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
y_train_predict_2

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [47]:
#測試集結果
y_test_predict=model.predict(x_test)
y_test_predict=y_test_predict[:,0]
y_test_predict=[np.round(i) for i in y_test_predict]
y_test_predict=np.array(y_test_predict)
from sklearn import metrics
print("精準度指標：")
print(metrics.classification_report(y_test,y_test_predict))
print("混淆矩陣：")
print(metrics.confusion_matrix(y_test,y_test_predict))

239/239 [==============================] - 0s 2ms/step
精準度指標：
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       927
           1       0.88      1.00      0.94      6697

    accuracy                           0.88      7624
   macro avg       0.44      0.50      0.47      7624
weighted avg       0.77      0.88      0.82      7624

混淆矩陣：
[[   0  927]
 [   0 6697]]


C:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

precision = precision_score(y_test,y_test_predict)
recall = recall_score(y_test,y_test_predict)
f1 = f1_score(y_test,y_test_predict)
roc_auc_score = roc_auc_score(y_test,y_test_predict)
print(precision)
print(recall)
print(f1)
print(roc_auc_score)

0.8784102833158447
1.0
0.9352698833880315
0.5


In [49]:
#驗證集結果
Y_va_predict=model.predict(X_va)
Y_va_predict=Y_va_predict[:,0]
Y_va_predict=[np.round(i) for i in Y_va_predict]
Y_va_predict=np.array(Y_va_predict
                     )
from sklearn import metrics
print("精準度指標：")
print(metrics.classification_report(Y_va,Y_va_predict))
print("混淆矩陣：")
print(metrics.confusion_matrix(Y_va,Y_va_predict))

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_va,Y_va_predict)
precision = precision_score(Y_va,Y_va_predict)
recall = recall_score(Y_va,Y_va_predict)
f1 = f1_score(Y_va,Y_va_predict)
roc_auc_score = roc_auc_score(Y_va,Y_va_predict)

print(accuracy)
print(precision)
print(recall)
print(f1)

In [ ]:
y_predict

In [ ]:
y_va_predict=model.predict(X_va)
y_va_predict=y_va_predict[:,0]
y_va_predict=[int(i) for i in y_va_predict]
y_va_predict=np.array(y_va_predict)
from sklearn import metrics
print("精準度指標：")
print(metrics.classification_report(Y_va,y_va_predict))
print("混淆矩陣：")
print(metrics.confusion_matrix(Y_va,y_va_predict))

In [ ]:

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18,6))
axs[0].plot(history.history['loss'], color='b', label='Training loss')
axs[0].plot(history.history['val_loss'], color='r', label='Validation loss')
axs[0].set_title("Loss curves")
axs[0].legend(loc='best', shadow=True)
axs[1].plot(history.history['accuracy'], color='b', label='Training accuracy')
axs[1].plot(history.history['val_accuracy'], color='r', label='Validation accuracy')
axs[1].set_title("Accuracy curves")
axs[1].legend(loc='best', shadow=True)

In [ ]:
################################

In [ ]:
# ＃生成LSTM网络
model = Sequential() 
model.add(LSTM(4,input_shape =(1,23)))
model.fit(x_train,y_train,epochs = 100,batch_size = 1,verbose = 2)
 
# ＃生成预测
trainpred = model.predict(x_train)
 
# ＃将标准化后的数据转换为原始数据
trainpred = scaler.inverse_transform(trainpred)
 
# ＃计算 RMSE
trainScore = math.sqrt(mean_squared_error(y_train [0],trainpred [:,0]))
 
# ＃训练预测
trainpredPlot = np.empty_like(dataset)
 
# ＃测试预测
 
# ＃绘制所有预测
inversetransform= plt.plot(scaler.inverse_transform(dataset))

In [ ]:
 # 將資料組成變好看一點
x_train = np.reshape(x_train, (x_train.shape[0],1,X.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,X.shape[1]))

In [ ]:
print("X_train size: {}".format(x_train.shape))
print("y_train size: {}".format(y_train.shape))
print("X_test size: {}".format(x_test.shape))
print("y_test size: {}".format(y_test.shape))

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential  #打開空白的網路神經網路機

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM


model = Sequential() 
# model.add(Dense(8, input_shape=(x_train.shape[0], x_train.shape[1])))

model.add(LSTM(8, input_shape=(1,23),activation="relu",return_sequences=True)) 
# model.add(LSTM(84, return_sequences= True)) 
# model.add(Dropout(0.2))
model.add(LSTM(8,activation="sigmoid"))
model.add(Dropout(0.5)) # Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.5
#model.add(LSTM(100,return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(50))
#model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))
# from keras.optimizers import SGD
model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy']) 
#以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)
model.summary()

#3*(32+1)
# model = Sequential()model.add(TimeDistributed(Dense(8), input_shape=(10, 16)))
# 输出还是10个向量，但是输出的维度由16变成了8，也就是（32,10,8）
#batch_size: 批尺寸，一次喂给lstm的数据量
#time_size: 时间步长，序列数据的长度
#feature_size: 特征维度，每个时间点输入数据的维度

In [ ]:
history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size,validation_data= (x_test, y_test),verbose=2)
score, acc = model.evaluate(x_test, y_test)



In [ ]:
###測試資料

In [ ]:
import numpy 
import random
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
df = pd.DataFrame({'N' : ['a', 'a', 'a','a','a','b','b','b','b','b','b','c','c','c','c','c','c','c','c','d','d','d','d','d','e','e','e','e'],
                   'year':['200903','200906','200909','200912','201003','200903','200906','200909','200912','201003','201006',
                           '200903','200906','200909','200912','201003','201006','201009','200812','200903','200906','200909',
                           '200912','201003','201012','200903','200906','200909'],
                   'Y' : [1,1,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1]})
df

In [ ]:
x = 4
n = 1
o = 0
t = 0
tt = 0

df["L"]=0
df["D"]=0
for name, group in df.groupby('N'):      
    if(numpy.any(group['Y']!= 0) ):
        if(len(group)-(x-1)<0):
            group ["L"]=0
        else:
            
            for i in range(t,t+len(group)-x+1):    
                if (numpy.all(group[i-t:i+n-t]['Y']==1)):
                    group.at [i,"L"]=1
                else:
                    group.at [i,"L"]=0                         
    else:

        if(len(group)-(x-1)<0):
            group ["L"]= 0
        else:
            for i in range(t,t+len(group)-x+1):    
                if (numpy.all(group[i-t:i+x-t]['Y']==0)):
                    group.at [i,"L"]= 1
                else:
                    group.at [i,"L"]= 0    
    df[t:t+len(group)]=group[:len(group)]        
    t+=len(group)

for name, group in df.groupby('N'):
    for d in range(len(group)):
        va_a = group.Y[d:d+1]
        if va_a.loc[d+o] == 1 or 2:
#             print(d+3)
#             print(group)
            if d+3 < len(group):
                for j in range(d+1,d+4):
                    va_l = group.L[j:j+1]
#                     print(j+t)
                    if va_l.loc[j+o] == 0:
                        group.at [j+o,"L"]= 2
    df[o:o+len(group)]=group[:len(group)] 
    o = o+len(group) 
    
print (df)       

#     print(t+=len(group))
    
    


In [ ]:
########################################111111
window_size = 3
series_s = df.copy()
# gg=0


indexNames = df[ df['L'] != 1 ].index
df.drop(indexNames , inplace=True)

# for mm,group in df.groupby('N'):
#     for kk in range(len(group)):
for ii in range(window_size):
    df = pd.concat([series_s.shift((ii-1)),df[['L','D']]], axis = 1)
                
df.dropna(axis=0, inplace=True)
#             df[gg:gg+len(group)]=group[:len(group)]
#     gg+=len(group) 
df
#     df.dropna(axis=0, inplace=True)
# group